### TORCH.AUTOGRAD 에 대한 간단한 소개
torch.autograd 는 신경망 학습을 지원하는 PyTorch의 자동 미분 엔진입니다. 

(torch.autograd 는 벡터-야코비안 곱을 계산하는 엔진입니다. )


이 단원에서는 autograd가 신경망 학습을 어떻게 돕는지에 대한 개념적 이해를 할 수 있습니다.

### 배경(Background)


신경망(NN; Neural Network)은 어떤 입력 데이터에 대해 실행되는 중첩(nested)된 함수들의 모음(collection)입니다. 

이 함수들은 PyTorch에서 Tensor로 저장되는, (가중치(weight)와 편향(bias)로 구성된) 매개변수들로 정의됩니다.

신경망을 학습하는 것은 2단계로 이루어집니다:

순전파(Forward Propagation): 순전파 단계에서, 신경망은 정답을 맞추기 위해 최선의 추측(best guess)을 합니다. 이렇게 추측을 하기 위해서 입력 데이터를 각 함수들에서 실행합니다.

역전파(Backward Propagation): 역전파 단계에서, 신경망은 추측한 값에서 발생한 오류(error)에 비례하여(proportionate) 매개변수들을 적절히 조절(adjust)합니다. 출력(output)로부터 역방향으로 이동하면서 오류에 대한 함수들의 매개변수들의 미분값( 변화도(gradient) )을 수집하고, 경사하강법(gradient descent)을 사용하여 매개변수들을 최적화 합니다. 역전파에 대한 자세한 설명은 [3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8)의 비디오 를 참고하세요.

### PyTorch에서 사용법
학습 단계를 하나만 살펴보겠습니다. 여기에서는 torchvision 에서 미리 학습된 resnet18 모델을 불러옵니다. 3채널짜리 높이와 넓이가 64인 이미지 하나를 표현하는 무작위의 데이터 텐서를 생성하고, 이에 상응하는 label(정답) 을 무작위 값으로 초기화합니다. 미리 학습된 모델의 정답(label)은 (1, 1000)의 모양(shape)을 갖습니다.

> 이 튜토리얼은 (텐서를 CUDA로 이동하더라도) GPU에서는 동작하지 않으며 CPU에서만 동작합니다.

In [2]:
import torch
from torchvision.models import resnet18, ResNet18_Weights
model = resnet18(weights=ResNet18_Weights.DEFAULT)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/baron/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
52.1%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100.0%


다음으로, 입력(input) 데이터를 모델의 각 층(layer)에 통과시켜 예측값(prediction)을 생성해보겠습니다. 이것이 순전파 단계 입니다.

In [3]:
prediction = model(data) # 순전파 단계(forward pass)

모델의 예측값과 그에 해당하는 정답(label)을 사용하여 오차(error, 손실(loss) )를 계산합니다. 다음 단계는 신경망을 통해 이 에러를 역전파하는 것입니다. 오차 텐서(error tensor)에 .backward() 를 호출하면 역전파가 시작됩니다. 그 다음 Autograd가 매개변수(parameter)의 .grad 속성(attribute)에, 모델의 각 매개변수에 대한 변화도(gradient)를 계산하고 저장합니다.

In [4]:
loss = (prediction - labels).sum()
loss.backward() # 역전파 단계(backward pass)

다음으로, 옵티마이저(optimizer)를 불러옵니다. 이 예제에서는 학습율(learning rate) 0.1과 모멘텀(momentum) 0.9를 갖는 SGD입니다. 옵티마이저(optimizer)에 모델의 모든 매개변수를 등록합니다.

In [5]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

마지막으로 .step 을 호출하여 경사하강법(gradient descent)을 시작합니다. 옵티마이저는 .grad 에 저장된 변화도에 따라 각 매개변수를 조정(adjust)합니다.

In [6]:
optim.step() # 경사하강법(gradient descent)

### Autograd에서 미분(differentiation)


autograd 가 어떻게 변화도(gradient)를 수집하는지 살펴보겠습니다. requires_grad=True 를 갖는 2개의 텐서(tensor) a 와 b 를 만듭니다. requires_grad=True 는 autograd 에 모든 연산(operation)들을 추적해야 한다고 알려줍니다.
https://tutorials.pytorch.kr/beginner/blitz/autograd_tutorial.html 이 링크 참고

In [1]:
import torch

a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

Q = 3*a**3 - b**2

In [2]:
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

In [3]:
# 수집된 변화도가 올바른지 확인합니다.
print(a.grad)
print(b.grad)

tensor([36., 81.])
tensor([-12.,  -8.])


torch.autograd 는 벡터-야코비안 곱을 계산하는 엔진입니다. 